In [1]:
import json
import requests
from bs4 import BeautifulSoup


In [2]:
def load_bookmarks(file_path):
    """Загружает содержимое файла закладок."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

In [3]:
def parse_html(bookmarks_html):
    """Парсит HTML содержимое и извлекает категории и ссылки."""
    soup = BeautifulSoup(bookmarks_html, 'html.parser')
    bookmarks = {}

    # Ищем все элементы закладок в HTML
    for dl in soup.find_all('dl'):
        category = dl.find('h3').text if dl.find('h3') else 'Без категории'
        links = {}
        
        # Ищем все ссылки в категории
        for dt in dl.find_all('dt'):
            a = dt.find('a')
            if a and 'href' in a.attrs:
                link_title = a.text
                link_url = a['href']
                links[link_title] = link_url
        
        bookmarks[category] = links  # Добавляем найденные ссылки к категории
    return bookmarks

In [4]:
def check_links(bookmarks):
    """Проверяет действительность каждой ссылки и возвращает только работающие ссылки."""
    valid_bookmarks = {}
    
    for category, links in bookmarks.items():
        valid_links = {}
        for title, url in links.items():
            try:
                # Отправляем HTTP-запрос к ссылке
                response = requests.head(url, allow_redirects=True, timeout=5)
                # Проверяем, что статус-код в диапазоне 200-299
                if response.status_code // 100 == 2:
                    valid_links[title] = url  # Если ссылка действительна, добавляем её
            except requests.RequestException:
                # Игнорируем ошибки, связанные с недоступностью ссылок
                continue
        valid_bookmarks[category] = valid_links  # Сохраняем действительные ссылки по категориям
    return valid_bookmarks

In [5]:
def sort_bookmarks(valid_bookmarks):
    """Сортирует закладки по категориям и в алфавитном порядке. """
    sorted_bookmarks = {}
    for category, links in valid_bookmarks.items():
        sorted_bookmarks[category] = {k: links[k] for k in sorted(links.keys())}  # Сортируем ссылки по названию
    return sorted_bookmarks

In [7]:
def main():
    """Основная функция для запуска программы."""
    file_path = 'E:/Dev/Git/pet_projects-1/Bookmark sorting/bookmarks_22.03.2025.html'  # Укажите путь к вашему файлу закладок
    bookmarks_html = load_bookmarks(file_path)  # Загружаем закладки из файла
    bookmarks = parse_html(bookmarks_html)  # Парсим HTML для извлечения закладок
    valid_bookmarks = check_links(bookmarks)  # Проверяем действительность ссылок
    sorted_bookmarks = sort_bookmarks(valid_bookmarks)  # Сортируем валидные ссылки

    # Выводим результат в формате JSON с отступами
    print(json.dumps(sorted_bookmarks, indent=4, ensure_ascii=False))

In [8]:
if __name__ == "__main__":
    main()  # Запускаем программу

KeyboardInterrupt: 